<a href="https://colab.research.google.com/github/lfernandodiniz/alura_vox_challengedatascience/blob/master/semana3/semana3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tendo feito o tratamento e análise dos dados, agora podemos construir modelos de classificação que serão úteis para o Alura Voz, entre os modelos que podemos criar estão o SVC, Decision Tree e Random Forest.

### Importando bibliotecas

Para a aplicação utilizaremos `pandas`, `seaborn`, `sklearn`, `imblearn` e `sys`. Para saber mais sobre as bibliotecas sklearn e imblearn, acesse a documentação:
- [Scikit Learn](https://scikit-learn.org/stable/); e
- [Imbalanced Learn](https://imbalanced-learn.org/stable/).